# Time-Frequency Analysis

#### The objective is to meticulously explore and dissect the intricate temporal and spectral characteristics of neural signals using advanced statistical methods. Through the utilization of the process_statsmatrix(statsmatrix) function and the subsequent visualization with plot_conditions(...), we find significant channels and significant time points for relax vs push conditions across emotionally charged stimuli.

In [1]:
import os
from os.path import join

import numpy as np 

from scipy.io import savemat, loadmat
from scipy.stats import sem

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import mne
from mne.time_frequency import tfr_morlet
from mne.baseline import rescale


### Load data for all conditions

In [ ]:
positive_relax = np.load('../../data/Osheen_TF_data/relax_positive.npy', allow_pickle=True)
positive_push = np.load('../../data/Osheen_TF_data/push_positive.npy', allow_pickle=True)

negative_relax = np.load('../../data/Osheen_TF_data/relax_negative.npy', allow_pickle=True)
negative_push = np.load('../../data/Osheen_TF_data/push_negative.npy', allow_pickle=True)

neutral_relax = np.load('../../data/Osheen_TF_data/relax_neutral.npy', allow_pickle=True)
neutral_push = np.load('../../data/Osheen_TF_data/push_neutral.npy', allow_pickle=True)

relax_all = np.load('../../data/Osheen_TF_data/relax_all.npy', allow_pickle=True)
push_all = np.load('../../data/Osheen_TF_data/push_all.npy', allow_pickle=True)

time_vector = np.load('../../data/Osheen_TF_data/time_vector.npy', allow_pickle=True)

channels_list = ["Fp1","AF7", "AF3", "F1", "F3", "F5", "F7", "FT7", "FC5", "FC3", "FC1", "C1", "C3", "C5", "T7", "TP7", "CP5", "CP3", "CP1", "P1", "P3", 
                "P5","P7","P9","PO7","PO3","O1","Iz","Oz","POz","Pz","CPz","Fpz","Fp2","AF8","AF4","AFz","Fz","F2","F4","F6","F8","FT8","FC6","FC4","FC2","FCz",
                "Cz","C2","C4","C6","T8","TP8","CP6","CP4","CP2","P2","P4","P6","P8","P10", "PO8", "PO4", "O2"]

### Convert to MATLAB files

In [ ]:
savemat('../../data/Osheen_TF_data/relax_positive.mat', {'data': positive_relax})
savemat('../../data/Osheen_TF_data/push_positive.mat', {'data': positive_push})

savemat('../../data/Osheen_TF_data/relax_negative.mat', {'data': negative_relax})
savemat('../../data/Osheen_TF_data/push_negative.mat', {'data': negative_push})

savemat('../../data/Osheen_TF_data/relax_neutral.mat', {'data': neutral_relax})
savemat('../../data/Osheen_TF_data/push_neutral.mat', {'data': neutral_push})

savemat('../../data/Osheen_TF_data/relax_all.mat', {'data': relax_all})
savemat('../../data/Osheen_TF_data/push_all.mat', {'data': push_all})

### Load statsmatrix for all conditions

In [ ]:
statsmatrix_positive = loadmat('../../data/Osheen_TF_data/posstatsmatrix_positive.mat')['statsmatrix_pos']
statsmatrix_negative = loadmat('../../data/Osheen_TF_data/posstatsmatrix_negative.mat')['statsmatrix_pos']
statsmatrix_neutral = loadmat('../../data/Osheen_TF_data/posstatsmatrix_neutral.mat')['statsmatrix_pos']
statsmatrix_all = loadmat('../../data/Osheen_TF_data/posstatsmatrix_all.mat')['statsmatrix_pos']

# Topography

### Plot topographical maps

In [ ]:
tstart = 1 # in seconds
tend = 3

tstart_idx = np.where(time_vector >= tstart)[0][0]
tend_idx = np.where(time_vector >= tend)[0][0]

x_alpha = negative_push[:, :, 0, tstart_idx:tend_idx].mean(axis=0) - negative_relax[:, :, 0, tstart_idx:tend_idx].mean(axis=0) # difference in alpha and beta in sig. time-window only to highlight the difference in topo plot

info = mne.create_info(ch_names=channels_list, sfreq=(negative_push.shape[-1]/5), ch_types='eeg') # make the new info object
montage = mne.channels.make_standard_montage('biosemi64') # apply biosemi64 montage
info.set_montage(montage)

topo_evoked = mne.EvokedArray(x_alpha[:, None, :].mean(axis=-1), info, tmin=0, nave=negative_push.shape[0]) # average across time to show power for each channel
mask = np.zeros(shape=(64, 1), dtype=bool)
sig_channels = np.unique(np.where(statsmatrix_negative == 1)[0]) # 1 is the sig cluster (statsmatrix is from matlab)
mask[sig_channels] = True

fig, ax_topo = plt.subplots(1, 1, figsize=(12, 5))
f = topo_evoked.plot_topomap(times=0, mask=mask, axes=ax_topo, cmap='Blues',
                            show=False,
                            colorbar=False, mask_params=dict(markersize=10), contours=4)
image = ax_topo.images[0]
    
# create additional axes (for ERF and colorbar)
divider = make_axes_locatable(ax_topo)

# add axes for colorbar
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(image, cax=ax_colorbar)
ax_topo.set_xlabel('Averaged Response')

### Find Significant Time Points and Channels across Alpha and Beta

##### This function is meticulously designed to process the statistical matrix statsmatrix, which encapsulates information about neural activity across various channels and time points. The ultimate goal of this function is to identify significant neural activity patterns in specific frequency bands (alpha and beta) and extract relevant data for subsequent visualization and analysis.

In [ ]:

def process_statsmatrix(statsmatrix):

    alpha_ch = []
    beta_ch = []
    
    for n in range(0, statsmatrix.shape[0]):
        ch_alpha = np.where(statsmatrix[n, 0, :] == 1)[0] # alpha
        if ch_alpha.shape[0] > 0:
            alpha_ch.append(n)

        ch_beta = np.where(statsmatrix[n, 1, :] == 1)[0] # beta
        if ch_beta.shape[0] > 0:
            beta_ch.append(n)

    sig_time_alpha = []
    unique_sig_time_alpha = []

    sig_time_beta = []
    unique_sig_time_beta = []
    
    for n in range(0, statsmatrix.shape[0]):
        sig_ch_alpha = np.where(statsmatrix[n, 0, :] == 1)[0] # alpha
        sig_ch_beta = np.where(statsmatrix[n, -1, :] == 1)[0] # beta

        if sig_ch_alpha.shape[0] > 0:
            sig_time_alpha.append(sig_ch_alpha)
            sig_time_beta.append(sig_ch_beta)

            temp_alpha = np.concatenate(sig_time_alpha)
            temp_beta = np.concatenate(sig_time_beta)

            unique_sig_time_alpha.append(np.unique(temp_alpha))
            if len(unique_sig_time_alpha) > 1:
                unique_sig_time_alpha.pop(0)

            unique_sig_time_beta.append(np.unique(temp_beta))
            if len(unique_sig_time_beta) > 1:
                unique_sig_time_beta.pop(0)
    
    return unique_sig_time_alpha, unique_sig_time_beta, alpha_ch, beta_ch



### Plot Alpha and Beta plots

In [ ]:
def plot_conditions(relax_condition, push_condition, plot_title, unique_sig_time_alpha, unique_sig_time_beta, alpha_ch, beta_ch):

    # Define the time range for plotting
    tstart = 1  # in seconds
    tend = 3

    # Find the indices corresponding to the time range
    tstart_idx = np.where(time_vector >= tstart)[0][0]
    tend_idx = np.where(time_vector >= tend)[0][0]

    # Calculate mean alpha values for relax and push conditions
    vi_alpha_relax = relax_condition[:, alpha_ch, 0, :].mean(axis=1)
    vi_alpha_push = push_condition[:, alpha_ch, 0, :].mean(axis=1)

    vi_beta_relax = relax_condition[:, beta_ch, 1, :].mean(axis=1)
    vi_beta_push = push_condition[:, beta_ch, 1, :].mean(axis=1)

    # Calculate standard error of the mean (SEM) for alpha values
    vi_sem_relax_alpha = sem(vi_alpha_relax, axis=0)
    vi_sem_push_alpha = sem(vi_alpha_push, axis=0)

    vi_sem_relax_beta = sem(vi_beta_relax, axis=0)
    vi_sem_push_beta = sem(vi_beta_push, axis=0)

    # print(vi_sem_relax_alpha)
    # print(vi_sem_push_alpha)

    # print(vi_sem_relax_beta)
    # print(vi_sem_push_beta)

    #Alpha Plot
    
    plt.figure(figsize=(10.5, 5))
    # Plot mean alpha values for push condition with shaded region
    plt.plot(time_vector, vi_alpha_push.mean(axis=0), color='#aecdc2', label='Push alpha')
    plt.fill_between(time_vector,
                     vi_alpha_push.mean(axis=0) + vi_sem_push_alpha,
                     vi_alpha_push.mean(axis=0) - vi_sem_push_alpha,
                     color='#aecdc2', alpha=0.4)

     # Plot mean alpha values for relax condition with shaded region
    plt.plot(time_vector, vi_alpha_relax.mean(axis=0), color='#3e8f8c', label='Relax alpha')
    plt.fill_between(time_vector,
                     vi_alpha_relax.mean(axis=0) + vi_sem_relax_alpha,
                     vi_alpha_relax.mean(axis=0) - vi_sem_relax_alpha,
                     color='#3e8f8c', alpha=0.4)

    # Add a vertical line at a specific time point
    plt.axvline(time_vector[128], c='k')

    plt.legend(loc='upper left')
    plt.ylabel('ALPHA')

    # Add horizontal bars to the plot
    bar_height = np.min(vi_alpha_relax) * 0.30  # Height of the horizontal bar (5% of max voltage)
    for index in unique_sig_time_alpha[0]:
        plt.barh(np.min(vi_alpha_relax), width=0.005, left=time_vector[tstart_idx+index-1], height=bar_height, color='green', alpha=0.2)
    
    print(time_vector[tstart_idx+unique_sig_time_alpha[0][0]], time_vector[tstart_idx+unique_sig_time_alpha[0][-1]])

    plt.savefig(f'Alpha_{plot_title}_plot.svg')


    #Beta Plot

    # Plot mean alpha values for push condition with shaded region
    plt.figure(figsize=(9, 5))
    plt.plot(time_vector, vi_beta_push.mean(axis=0), color='#eaafc3', label='Push beta')
    plt.fill_between(time_vector,
                     vi_beta_push.mean(axis=0) + vi_sem_push_beta,
                     vi_beta_push.mean(axis=0) - vi_sem_push_beta,
                     color='#eaafc3', alpha=0.4)

     # Plot mean beta values for relax condition with shaded region
    plt.plot(time_vector, vi_beta_relax.mean(axis=0), color='#cd5d8a', label='Relax beta')
    plt.fill_between(time_vector,
                     vi_beta_relax.mean(axis=0) + vi_sem_relax_beta,
                     vi_beta_relax.mean(axis=0) - vi_sem_relax_beta,
                     color='#cd5d8a', alpha=0.4)


    # Add a vertical line at a specific time point
    plt.axvline(time_vector[128], c='k')

    # Add legends to the plot
    plt.legend(loc='upper left')
    plt.xlabel('Time (s)')
    plt.ylabel('BETA')

    plt.tight_layout()  # Ensures proper spacing between subplots
    plt.savefig(f'Beta_{plot_title}_plot.svg')

    bar_height = np.min(vi_beta_relax) * 0.30  # Height of the horizontal bar (5% of max voltage)
    for index in unique_sig_time_beta[0]:
        plt.barh(np.min(vi_beta_relax), width=0.005, left=time_vector[tstart_idx+index-1], height=bar_height, color='green', alpha=0.2)
    
    print(time_vector[tstart_idx+unique_sig_time_beta[0][0]], time_vector[tstart_idx+unique_sig_time_beta[0][-1]])

    # Display the plot
    plt.show()



### Plot for Positively Charged Stimuli

In [ ]:
# Print plots
unique_sig_time_alpha_pos, unique_sig_time_beta_pos, alpha_ch_pos, beta_ch_pos = process_statsmatrix(statsmatrix_positive)
plot_conditions(positive_relax, positive_push, 'Positive', unique_sig_time_alpha_pos, unique_sig_time_beta_pos, alpha_ch_pos, beta_ch_pos)

np.save('../../data/Osheen_TF_data/unique_sig_time_alpha_pos.npy', unique_sig_time_alpha_pos)
np.save('../../data/Osheen_TF_data/unique_sig_time_beta_pos.npy', unique_sig_time_beta_pos)
np.save('../../data/Osheen_TF_data/alpha_ch_pos.npy', alpha_ch_pos)
np.save('../../data/Osheen_TF_data/beta_ch_pos.npy', beta_ch_pos)


### Plot for Negatively Charged Stimuli

In [ ]:
unique_sig_time_alpha_neg, unique_sig_time_beta_neg, alpha_ch_neg, beta_ch_neg = process_statsmatrix(statsmatrix_negative)
plot_conditions(negative_relax, negative_push, 'Negative', unique_sig_time_alpha_neg, unique_sig_time_beta_neg, alpha_ch_neg, beta_ch_neg)

np.save('../../data/Osheen_TF_data/unique_sig_time_alpha_neg.npy', unique_sig_time_alpha_neg)
np.save('../../data/Osheen_TF_data/unique_sig_time_beta_neg.npy', unique_sig_time_beta_neg)
np.save('../../data/Osheen_TF_data/alpha_ch_neg.npy', alpha_ch_neg)
np.save('../../data/Osheen_TF_data/beta_ch_neg.npy', beta_ch_neg)

### Plot for Neutrally Charged Stimuli

In [ ]:
unique_sig_time_alpha_neu, unique_sig_time_beta_neu, alpha_ch_neu, beta_ch_neu = process_statsmatrix(statsmatrix_neutral)
plot_conditions(neutral_relax, neutral_push, 'Neutral', unique_sig_time_alpha_neu, unique_sig_time_beta_neu, alpha_ch_neu, beta_ch_neu)

np.save('../../data/Osheen_TF_data/unique_sig_time_alpha_neu.npy', unique_sig_time_alpha_neu)
np.save('../../data/Osheen_TF_data/unique_sig_time_beta_neu.npy', unique_sig_time_beta_neu)
np.save('../../data/Osheen_TF_data/alpha_ch_neu.npy', alpha_ch_neu)
np.save('../../data/Osheen_TF_data/beta_ch_neu.npy', beta_ch_neu)